# Training a Flow Matching Model for Weather Prediction

This notebook demonstrates how to train a WeatherFlowMatch model for weather prediction using ERA5 data. We'll cover:

1. Setting up the model architecture
2. Configuring an effective training pipeline
3. Incorporating physics constraints
4. Implementing monitoring and visualization during training
5. Evaluating model performance
6. Saving and loading trained models

Let's get started!

## 1. Setup and Dependencies

In [ ]:
# Add repository root to Python path to find weatherflow package
import sys
import os
# Get absolute path to repo root
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
repo_root = os.path.abspath(os.path.join(notebook_dir, '..'))
# Add to path if not already there
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
print(f"Added {repo_root} to Python path")

# Add repository root to Python path
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))

# Install WeatherFlow if needed
try:
    import weatherflow
    print(f"WeatherFlow version: {weatherflow.__version__}")
except ImportError:
    !pip install -e ..
    import weatherflow
    print(f"WeatherFlow installed, version: {weatherflow.__version__}")

In [ ]:
# Mock dependencies that might not be availabletry:    import sys    notebook_dir = os.path.dirname(os.path.abspath('__file__'))    repo_root = os.path.abspath(os.path.join(notebook_dir, '..'))    mock_path = os.path.join(repo_root, 'mock_dependencies.py')        if os.path.exists(mock_path):        # Execute the mock dependencies script        with open(mock_path, 'r') as f:            mock_code = f.read()            # Add repo_root to sys.path if not already there            if repo_root not in sys.path:                sys.path.insert(0, repo_root)            # Execute the script            exec(mock_code)            # Call the function to install all mocks            exec("install_all_mocks()")    else:        print(f"Warning: Mock dependencies script not found at {mock_path}")except Exception as e:    print(f"Error loading mock dependencies: {str(e)}")

In [ ]:
# Import standard libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import os
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')  # Suppress some warnings for cleaner output

# Import WeatherFlow components
from weatherflow.data import ERA5Dataset, create_data_loaders
from weatherflow.models import WeatherFlowMatch, WeatherFlowODE
from weatherflow.utils import WeatherVisualizer
from weatherflow.training import FlowTrainer

# Set up matplotlib
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['figure.dpi'] = 100

# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 2. Configuration

Let's define our configuration parameters for the experiment.

In [ ]:
# Define experiment configuration
config = {
    # Data parameters
    "variables": ['z', 't', 'u', 'v'],  # Geopotential, temperature, u-wind, v-wind
    "pressure_levels": [500],  # 500 hPa level
    "train_years": ('2015', '2016'),  # Training period
    "val_years": ('2017', '2017'),  # Validation period
    "batch_size": 16,
    "num_workers": 4,
    "normalize": True,  # Apply normalization
    
    # Model parameters
    "hidden_dim": 128,
    "n_layers": 4,
    "use_attention": True,
    "physics_informed": True,
    
    # Training parameters
    "learning_rate": 1e-4,
    "weight_decay": 1e-5,
    "max_epochs": 20,  # Reduced for demonstration
    "early_stopping_patience": 5,
    "use_amp": True,  # Use mixed precision training if available
    
    # Experiment tracking
    "exp_name": f"flow_match_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
    "save_dir": "../models",
    "save_checkpoint_freq": 5,
    
    # Visualization
    "plot_freq": 5,
    "vis_dir": "../visualizations"
}

# Create output directories
os.makedirs(config["save_dir"], exist_ok=True)
os.makedirs(config["vis_dir"], exist_ok=True)
exp_vis_dir = os.path.join(config["vis_dir"], config["exp_name"])
os.makedirs(exp_vis_dir, exist_ok=True)

# Save config for reproducibility
with open(os.path.join(config["save_dir"], f"{config['exp_name']}_config.json"), 'w') as f:
    json.dump(config, f, indent=2)

## 3. Load and Prepare Data

Now let's load our ERA5 data for training and validation.

In [ ]:
print("Loading ERA5 data for training and validation...")

# Load training and validation data
train_loader, val_loader = create_data_loaders(
    variables=config["variables"],
    pressure_levels=config["pressure_levels"],
    train_slice=config["train_years"],
    val_slice=config["val_years"],
    batch_size=config["batch_size"],
    num_workers=config["num_workers"],
    normalize=config["normalize"]
)

print(f"Training samples: {len(train_loader.dataset)}")
print(f"Validation samples: {len(val_loader.dataset)}")

# Examine the data structure by looking at a batch
sample_batch = next(iter(train_loader))
print("\nSample batch structure:")
for key, value in sample_batch.items():
    if isinstance(value, dict):
        print(f"  - {key}: {type(value)}")
        for subkey, subval in value.items():
            print(f"      {subkey}: {type(subval)}")
    else:
        print(f"  - {key}: {type(value)}, shape: {value.shape}")

# Extract grid size from the data
input_channels = sample_batch['input'].shape[1]
grid_size = sample_batch['input'].shape[2:]  # (lat, lon)
print(f"\nInput channels: {input_channels}")
print(f"Grid size: {grid_size}")

# Visualize a sample from the training data
print("\nVisualizing a sample from the training data:")
visualizer = WeatherVisualizer()

# Extract the first sample for visualization
input_data = sample_batch['input'][0].cpu()  # First batch item
target_data = sample_batch['target'][0].cpu()
var_names = config["variables"]

# Visualize each variable
for i, var_name in enumerate(var_names):
    plt.figure(figsize=(14, 6))
    
    # Plot input state (current weather)
    

In [ ]:
    plt.subplot(1, 2, 1)
    plt.imshow(input_data[i, 0].numpy(), cmap=visualizer.VAR_CMAPS.get(var_name, 'viridis'))
    plt.colorbar()
    plt.title(f"{var_name} - Current")
    
    # Plot target state (future weather)
    plt.subplot(1, 2, 2)
    plt.imshow(target_data[i, 0].numpy(), cmap=visualizer.VAR_CMAPS.get(var_name, 'viridis'))
    plt.colorbar()
    plt.title(f"{var_name} - Future (6h later)")
    
    plt.tight_layout()
    plt.show()

## 4. Define the Model

Let's define our WeatherFlowMatch model architecture. Here, we'll use a configuration that includes attention mechanisms and physics-informed constraints.

In [ ]:
# Initialize the WeatherFlowMatch model
model = WeatherFlowMatch(
    input_channels=input_channels,
    hidden_dim=config['hidden_dim'],
    n_layers=config['n_layers'],
    use_attention=config['use_attention'],
    physics_informed=config['physics_informed']
)

# Move the model to the GPU if available
model = model.to(device)

# Print model architecture
print(model)

## 5. Training Setup

Now, we'll set up our training components, including the optimizer, learning rate scheduler, and loss function.

In [ ]:
# Configure the optimizer
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config['learning_rate'],
    weight_decay=config['weight_decay']
)

# Use a ReduceLROnPlateau learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    factor=0.5,  # Reduce LR by a factor of 0.5
    patience=3,   # Reduce LR after 3 epochs with no improvement
    verbose=True  # Print messages when LR is updated
)

# Define the loss function (MSE loss)
loss_fn = nn.MSELoss()

# Initialize the trainer
trainer = FlowTrainer(
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader,
    scheduler=scheduler,
    config=config,
    device=device
)

## 6. Training Loop

Let's start the training loop and monitor the training process.

In [ ]:
# Train the model
print("Starting training...")
trainer.train()

## 7. Visualizations and Monitoring

During training, we can create visualizations to monitor the model's progress.  For example, we can plot the loss over time or visualize model predictions.

In [ ]:
# Load training history
history_path = os.path.join(config['save_dir'], f"{config['exp_name']}_history.json")
with open(history_path, 'r') as f:
    history = json.load(f)

# Plotting training and validation loss
plt.figure(figsize=(12, 6))
plt.plot(history['train_loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Time')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(exp_vis_dir, 'loss_curves.png'))
plt.show()

# Load a sample from the validation set
sample_batch = next(iter(val_loader))
x0, x1 = sample_batch['input'].to(device), sample_batch['target'].to(device)
var_names = config['variables']

# Generate predictions with the trained model
model.eval()
with torch.no_grad():
    # Create ODE solver
    ode_model = WeatherFlowODE(
        flow_model=model,
        solver_method='dopri5',
        rtol=1e-4,
        atol=1e-4
    )

    t_eval = torch.tensor([1.0], device=device)  # Predict at t=1
    predictions = ode_model(x0, t_eval)
    y_pred = predictions[0].cpu()
    x1 = x1.cpu()

# Plotting predictions vs ground truth
num_samples = min(4, x0.shape[0])  # Number of samples to visualize
fig, axes = plt.subplots(num_samples, len(var_names) * 2, figsize=(20, 5 * num_samples))

for i in range(num_samples):
    for j, var_name in enumerate(var_names):
        # Ground truth plot
        ax = axes[i, j*2]
        im = ax.imshow(x1[i, j, 0].numpy(), cmap=visualizer.VAR_CMAPS.get(var_name, 'viridis'))
        ax.set_title(f'{var_name} - Ground Truth (6h later)')
        fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        
        # Prediction plot
        ax = axes[i, j*2 + 1]
        im = ax.imshow(y_pred[i, j, 0].numpy(), cmap=visualizer.VAR_CMAPS.get(var_name, 'viridis'))
        ax.set_title(f'{var_name} - Prediction (6h later)')
        fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

plt.tight_layout()
plt.savefig(os.path.join(exp_vis_dir, 'predictions_vs_ground_truth.png'))
plt.show()

## 8. Animate Predictions

We can also create animations of the model's predictions.

In [ ]:
print("Generating animation...")

import matplotlib.animation as animation

# Load a sample from the validation set
sample_batch = next(iter(val_loader))
x0 = sample_batch['input'][:1].to(device) # Single sample for animation

# Define lead times for the animation
lead_times = torch.linspace(0, 1.0, 100, device=device)

# Generate predictions at multiple lead times
model.eval()
with torch.no_grad():
    # Create ODE solver
    ode_model = WeatherFlowODE(
        flow_model=model,
        solver_method='dopri5',
        rtol=1e-4,
        atol=1e-4
    )
    predictions = ode_model(x0, lead_times)

# Animation function
def animate(i):
    imgs = []
    for j, var_name in enumerate(config['variables']):
        ax = axes[j]
        ax.clear()
        ax.set_title(f'{var_name} - Lead Time: {lead_times[i]:.2f}')
        im = ax.imshow(predictions[i, 0, j, 0].cpu().numpy(), cmap=visualizer.VAR_CMAPS.get(var_name, 'viridis'), animated=True)
        imgs.append([im])
    return imgs

# Create animation figure
fig, axes = plt.subplots(1, len(config['variables']), figsize=(15, 5))
fig.tight_layout()

# Generate animation
anim = animation.FuncAnimation(
    fig, animate, frames=len(lead_times), interval=100, blit=False, repeat=True
)

# Save animation (optional)
animation_path = os.path.join(exp_vis_dir, 'weather_prediction_animation.gif')
anim.save(animation_path, writer='imagemagick', fps=10)
print(f"Animation saved to {animation_path}")

# Display animation in the notebook
from IPython.display import HTML
HTML(anim.to_jshtml())

In [ ]:
## 9. Calculate Evaluation Metrics

print("Calculating evaluation metrics...")

# Function to calculate MSE, MAE, and bias for each variable
def calculate_metrics(predictions, ground_truth, var_names):
    """Calculate evaluation metrics."""
    metrics = {}
    
    # Overall metrics
    mse = ((predictions - ground_truth) ** 2).mean().item()
    mae = torch.abs(predictions - ground_truth).mean().item()
    bias = (predictions - ground_truth).mean().item()
    
    metrics['overall'] = {
        'mse': mse,
        'rmse': np.sqrt(mse),
        'mae': mae,
        'bias': bias
    }
    
    # Per-variable metrics
    for i, var in enumerate(var_names):
        var_pred = predictions[:, i]
        var_truth = ground_truth[:, i]
        
        var_mse = ((var_pred - var_truth) ** 2).mean().item()
        var_mae = torch.abs(var_pred - var_truth).mean().item()
        var_bias = (var_pred - var_truth).mean().item()
        
        metrics[var] = {
            'mse': var_mse,
            'rmse': np.sqrt(var_mse),
            'mae': var_mae,
            'bias': var_bias
        }
    
    return metrics

# Evaluate on the whole validation set
def evaluate_model(model, val_loader, device, config):
    """Evaluate model on the validation set."""
    # Create ODE solver
    ode_model = WeatherFlowODE(
        flow_model=model,
        solver_method='dopri5',
        rtol=1e-4,
        atol=1e-4
    )
    
    # Set model to evaluation mode
    model.eval()
    
    # Initialize metrics
    all_metrics = []
    
    # Evaluate on batches
    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(val_loader, desc="Evaluating")):
            # Get data
            x0, x1 = batch['input'].to(device), batch['target'].to(device)
            
            # Generate predictions
            t_eval = torch.tensor([1.0], device=device)  # Predict at t=1
            predictions = ode_model(x0, t_eval)
            
            # Extract predictions at t=1
            y_pred = predictions[0]  # Shape: [batch_size, channels, lat, lon]
            
            # Calculate metrics for this batch
            batch_metrics = calculate_metrics(y_pred.flatten(1), x1.flatten(1), config['variables'])
            all_metrics.append(batch_metrics)
            
            # Only process a subset of batches for speed if dataset is large
            if batch_idx >= 10:
                break
    
    # Aggregate metrics across batches
    aggregated_metrics = {
        'overall': {metric: np.mean([b['overall'][metric] for b in all_metrics]) 
                   for metric in ['mse', 'rmse', 'mae', 'bias']}
    }
    
    for var in config['variables']:
        aggregated_metrics[var] = {
            metric: np.mean([b[var][metric] for b in all_metrics]) 
            for metric in ['mse', 'rmse', 'mae', 'bias']
        }
    
    return aggregated_metrics

# Run evaluation
eval_metrics = evaluate_model(model, val_loader, device, config)

# Print evaluation results
print("\nEvaluation Metrics:")
print(f"Overall: RMSE = {eval_metrics['overall']['rmse']:.4f}, MAE = {eval_metrics['overall']['mae']:.4f}")

print("\nPer-Variable Metrics:")
for var in config['variables']:
    print(f"{var}: RMSE = {eval_metrics[var]['rmse']:.4f}, MAE = {eval_metrics[var]['mae']:.4f}, Bias = {eval_metrics[var]['bias']:.4f}")

# Save metrics to file
metrics_path = os.path.join(config['save_dir'], f"{config['exp_name']}_metrics.json")
with open(metrics_path, 'w') as f:
    # Convert numpy values to Python types for JSON serialization
    json_metrics = {}
    for key, val in eval_metrics.items():
        json_metrics[key] = {k: float(v) for k, v in val.items()}
    
    json.dump(json_metrics, f, indent=2)

print(f"\nMetrics saved to {metrics_path}")

In [ ]:
## 10. Compare with Baseline Predictions

print("\nComparing with climatology and persistence baselines...")

# Function to create baseline predictions
def create_baseline_predictions(val_loader, type='persistence'):
    """Create baseline predictions.
    
    Args:
        val_loader: Validation data loader
        type: Type of baseline ('persistence' or 'climatology')
        
    Returns:
        Dictionary of metrics
    """
    all_metrics = []
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Evaluating {type} baseline"):
            # Get data
            x0, x1 = batch['input'], batch['target']
            
            if type == 'persistence':
                # Persistence baseline: use current state as prediction
                y_pred = x0
            elif type == 'climatology':
                # Climatology baseline: use mean of training data
                # For simplicity, we'll use zeros (assuming normalized data)
                y_pred = torch.zeros_like(x1)
            else:
                raise ValueError(f"Unknown baseline type: {type}")
            
            # Calculate metrics
            batch_metrics = calculate_metrics(y_pred.flatten(1), x1.flatten(1), config['variables'])
            all_metrics.append(batch_metrics)
            
            # Only process a subset of batches for speed if dataset is large
            if len(all_metrics) >= 10:
                break
    
    # Aggregate metrics
    aggregated_metrics = {
        'overall': {metric: np.mean([b['overall'][metric] for b in all_metrics]) 
                   for metric in ['mse', 'rmse', 'mae', 'bias']}
    }
    
    for var in config['variables']:
        aggregated_metrics[var] = {
            metric: np.mean([b[var][metric] for b in all_metrics]) 
            for metric in ['mse', 'rmse', 'mae', 'bias']
        }
    
    return aggregated_metrics

# Calculate baseline metrics
persistence_metrics = create_baseline_predictions(val_loader, type='persistence')
climatology_metrics = create_baseline_predictions(val_loader, type='climatology')

# Print comparison results
print("\nModel vs Baseline RMSE Comparison:")
print(f"{{'Variable':<10}} {{'Model':<10}} {{'Persistence':<15}} {{'Climatology':<15}} {{'Improvement %':<15}}")
print("-" * 65)

for var in config['variables']:
    model_rmse = eval_metrics[var]['rmse']
    persistence_rmse = persistence_metrics[var]['rmse']
    climatology_rmse = climatology_metrics[[var]['rmse']
    
    # Calculate improvement over persistence
    improvement = (persistence_rmse - model_rmse) / persistence_rmse * 100
    
    print(f"{var:<10} {model_rmse:<10.4f} {persistence_rmse:<15.4f} {climatology_rmse:<15.4f} {improvement:<15.2f}")

# Create comparison plot
plt.figure(figsize=(14, 8))

# Set up variables and metrics for plotting
vars_for_plot = config['variables']
model_rmse = [eval_metrics[var]['rmse'] for var in vars_for_plot]
persistence_rmse = [persistence_metrics[var]['rmse'] for var in vars_for_plot]
climatology_rmse = [climatology_metrics[var]['rmse'] for var in vars_for_plot]

# Create bar positions
x = np.arange(len(vars_for_plot))
width = 0.25

# Plot bars
plt.bar(x - width, model_rmse, width, label='Flow Matching Model')
plt.bar(x, persistence_rmse, width, label='Persistence Baseline')
plt.bar(x + width, climatology_rmse, width, label='Climatology Baseline')

# Customize plot
plt.xlabel('Variable')
plt.ylabel('RMSE')
plt.title('Model vs Baseline RMSE Comparison')
plt.xticks(x, vars_for_plot)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Save plot
plt.savefig(os.path.join(exp_vis_dir, 'baseline_comparison.png'))
plt.show()

In [ ]:
## 11. Model Usage Example

print("\nDemonstrating how to use the trained model for prediction...")

# Function to demonstrate model usage
def demonstrate_model_usage(model, input_data, device):
    """Show how to use the trained model for prediction."""
    # Create ODE solver with the trained model
    ode_model = WeatherFlowODE(
        flow_model=model,
        solver_method='dopri5',  # Options: 'euler', 'midpoint', 'rk4', 'dopri5'
        rtol=1e-4,
        atol=1e-4
    )
    
    # Move data to device
    x0 = input_data.to(device)
    
    # Define multiple lead times (in fractions of the prediction interval)
    # 0.0 = current state, 1.0 = full prediction interval (e.g., 6 hours)
    lead_times = torch.linspace(0, 1.0, 5, device=device)
    
    # Generate predictions
    with torch.no_grad():
        predictions = ode_model(x0, lead_times)
    
    return predictions, lead_times

# Get a sample from validation data
sample_batch = next(iter(val_loader))
input_data = sample_batch['input'][:1]  # Just use first sample

# Run demonstration
print("Generating predictions at multiple lead times...")
predictions, lead_times = demonstrate_model_usage(model, input_data, device)

print(f"Generated predictions with shape: {predictions.shape}")
print(f"Lead times: {lead_times.cpu().numpy()}")

# Show code example
print("""
## Example Code for Using the Trained Model

```python
from weatherflow.models import WeatherFlowMatch, WeatherFlowODE
import torch

# Load model
model = WeatherFlowMatch(
    input_channels=4,
    hidden_dim=128,
    n_layers=4,
    use_attention=True,
    physics_informed=True
)

# Load weights
checkpoint = torch.load('path/to/model_checkpoint.pt')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Create ODE solver
ode_model = WeatherFlowODE(
    flow_model=model,
    solver_method='dopri5',
    rtol=1e-4,
    atol=1e-4
)

# Prepare input data
input_data = torch.tensor(your_input_data)

# Define lead times
lead_times = torch.linspace(0, 1.0, 5)

# Generate predictions
with torch.no_grad():
    predictions = ode_model(input_data, lead_times)

# predictions.shape: [n_lead_times, batch_size, channels, lat, lon]
```
""")

In [ ]:
## 12. Conclusion

print("""
## Conclusion

In this notebook, we've trained a Flow Matching model for weather prediction:

1. We set up the WeatherFlowMatch model with physics-informed constraints
2. We trained the model on ERA5 data and monitored its progress
3. We evaluated the model's performance and compared it with baselines
4. We demonstrated how to use the trained model for prediction

Key findings:
- The model outperforms persistence and climatology baselines
- Physics-informed constraints help maintain physical consistency
- The flow matching approach allows for predictions at arbitrary lead times

Next steps:
- Experiment with different model architectures and hyperparameters
- Train on longer time periods and more variables
- Evaluate on additional test sets and metrics
- Compare with other state-of-the-art weather prediction models

For detailed evaluation on the WeatherBench2 benchmark, see the next notebook.
""")

# Save trained model for future use
final_model_path = os.path.join(config['save_dir'], f"{config['exp_name']}_final.pt")
torch.save({
    'model_state_dict': model.state_dict(),
    'config': config,
    'metrics': eval_metrics
}, final_model_path)

print(f"Final model saved to {final_model_path}")